In [ ]:
!pip install opencv-python
!pip install face_recognition
!pip install numpy
!pip install mediapipe opencv-python face_recognition
!pip install Pillow numpy opencv-python albumentations
!pip install opencv-python-headless face-recognition dlib matplotlib
!pip install tqdm
!pip install requests pillow
!pip install pylance
!pip install torch torchvision
!pip install pandas requests beautifulsoup4
# (Optional: install dlib separately if face_recognition does not handle it)


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import face_recognition
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)


# Load known faces
def load_known_faces(known_faces_dir):
    known_face_encodings = []
    known_face_names = []
    for filename in os.listdir(known_faces_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(known_faces_dir, filename)
            image = face_recognition.load_image_file(image_path)
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(os.path.splitext(filename)[0])
            else:
                print(f"No face found in {filename}. Skipping.")
    return known_face_encodings, known_face_names


# Register a new student by capturing 10 images
def register_student(known_faces_dir):
    student_name = input("Enter the name of the new student: ")
    video_capture = cv2.VideoCapture(0)
    print("Capturing images for", student_name)
    images_captured = 0

    while images_captured < 10:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            img_path = os.path.join(known_faces_dir, f"{student_name}.jpg")
            cv2.imwrite(img_path, frame)
            images_captured += 1
            print(f"Captured {images_captured} images for {student_name}")

        plt.imshow(rgb_frame)
        display(plt.gcf())
        clear_output(wait=True)

    video_capture.release()
    print(f"Finished registering {student_name}. You can now take attendance.")


# Take attendance by recognizing known faces
def take_attendance(known_face_encodings, known_face_names):
    video_capture = cv2.VideoCapture(0)
    attendance = set()

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)
        face_locations = []

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = rgb_frame.shape
                x = int(bboxC.xmin * w)
                y = int(bboxC.ymin * h)
                width = int(bboxC.width * w)
                height = int(bboxC.height * h)
                face_locations.append((y, x + width, y + height, x))

        if face_locations:
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                if True in matches:
                    best_match_index = np.argmin(face_recognition.face_distance(known_face_encodings, face_encoding))
                    name = known_face_names[best_match_index]
                    attendance.add(name)

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        display(plt.gcf())
        clear_output(wait=True)

        action = input("Type 'end' to stop attendance or press Enter to continue: ").lower()
        if action == 'end':
            break

    plt.close()
    video_capture.release()
    print("Attendance taken for:", attendance)


if __name__ == "__main__":
    known_faces_dir = 'Class_Attendance/Students'

    while True:
        action = input("Type 'register' to register a new student or 'attendance' to take attendance (or 'exit' to quit): ").lower()
        
        if action == 'register':
            register_student(known_faces_dir)
        elif action == 'attendance':
            known_face_encodings, known_face_names = load_known_faces(known_faces_dir)
            take_attendance(known_face_encodings, known_face_names)
        elif action == 'exit':
            print("Exiting program.")
            break
        else:
            print("Invalid action. Please try again.")


In [ ]:
"""

1. Load Known Faces :
Time Complexity: O(n) - where n is the number of images in the directory.
Space Complexity: O(n) - for storing face encodings and names.

2. Register New Student :
Time Complexity: O(1) - fixed number of images 10 captured, but processing each frame can vary.
Space Complexity: O(1) - for temporary variables and images captured.

3. Take Attendance :
Time Complexity: O(d . n) - where d is the number of detected faces, and n is the number of known faces.
Space Complexity: O(d) - for storing detected face locations and encodings.

Overall : 
Overall Time Complexity: Dominated by attendance function, O(d . n).
Overall Space Complexity: O(n) - primarily for known face encodings.

"""